In [1]:
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.core.bbox_utils import convert_bbox_to_albumentations
from ultralytics.utils.plotting import Annotator

ERROR:albumentations.check_version:Error fetching version info
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/urllib/request.py", line 1344, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1336, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1382, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1331, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1091, in _send_output
    self.send(msg)
  File "/Library/Frameworks/Python.framework/

In [2]:
dataset_width = 640
dataset_height = 512
classes = {
  0: 'bike',
  1: 'car',
  2: 'light',
  3: 'motor',
  4: 'other vehicle',
  5: 'person',
  6: 'rider',
  7: 'sign',
  8: 'truck'
}

In [3]:
train_dataset_path = 'filtered-data/images/train'
train_images = sorted(os.listdir(train_dataset_path))
train_images_paths = [os.path.join(train_dataset_path, image) for image in train_images if image.endswith('.jpg')]

In [5]:
def denormalize(x, y, w, h, image_width, image_height):
    denorm_x = x * image_width
    denorm_y = y * image_height
    denorm_w = w * image_width
    denorm_h = h * image_height
    return denorm_x, denorm_y, denorm_w, denorm_h

def normalize(x, y, w, h, image_width, image_height):
    norm_x = x / image_width
    norm_y = y / image_height
    norm_w = w / image_width
    norm_h = h / image_height
    return norm_x, norm_y, norm_w, norm_h

In [6]:
def load_bboxes(labels_path):
    bboxes = []
    with open(labels_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            id_class = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            bboxes.append((id_class, x_center, y_center, width, height))
    return bboxes

In [7]:
def plot_image(image, bboxes):
  image_height, image_width, _ = image.shape
  annotator = Annotator(image)
  for bbox in bboxes:
      id_class, x, y, w, h = bbox
      class_name = classes[id_class]
      x, y, w, h = denormalize(x, y, w, h, image_width, image_height)
      
      x1 = int(x - w / 2)
      y1 = int(y - h / 2)
      x2 = int(x + w / 2)
      y2 = int(y + h / 2)

      annotator.box_label([x1,y1,x2,y2], class_name)
    
  plt.figure(figsize=(10, 10))
  plt.imshow(annotator.result())
  plt.axis('off')
  plt.show()

In [8]:
#for image_path in train_images_paths:
  #image = cv2.imread(image_path)
  #labels_path = image_path.replace("images", "labels").replace(".jpg", ".txt")
  #bboxes = load_bboxes(labels_path)
  #plot_image(image, bboxes)

In [9]:
def fix_albumentation_range_errors(alb_bboxes):
  if len(alb_bboxes) == 0:
    return []
  for index, bbox in enumerate(alb_bboxes):
    try:
      convert_bbox_to_albumentations(bbox=bbox, source_format='yolo', rows=512, cols=dataset_width, check_validity=True)
      #print("OK with: ", bbox)
    except:
      print("Error with: ",bbox)
      accepted = False
      xn,yn,wn,hn = bbox
      while accepted is False:
        x,y,w,h = denormalize(xn,yn,wn,hn,dataset_width,dataset_height)
        w -= 1
        h -= 1
        xn,yn,wn,hn = normalize(x,y,w,h,dataset_width,dataset_height)
        try:
          convert_bbox_to_albumentations(bbox=[xn,yn,wn,hn], source_format='yolo', rows=dataset_height, cols=dataset_width, check_validity=True)
          accepted = True
        except:
          accepted = False
      print("Fixed with: ",xn,yn,wn,hn)
      alb_bboxes[index] = [xn,yn,wn,hn]
  return alb_bboxes

In [10]:
def alb_bboxes_to_bboxes(alb_bboxes, alb_class_labels):
  bboxes = []
  for index, alb_bbox in enumerate(alb_bboxes):
    x,y,w,h = alb_bbox
    class_name = alb_class_labels[index]
    for id, name in classes.items():
        if name == class_name:
             class_id = id
    bboxes.append((class_id, x,y,w,h))
  return bboxes

def bboxes_to_alb_bboxes(bboxes):
  alb_bboxes = []
  alb_class_labels = []
  for bbox in bboxes:
    id_class, xn, yn, wn, hn = bbox
    class_name = classes[id_class]
    alb_bboxes.append([xn, yn, wn, hn])
    alb_class_labels.append(class_name)
  return alb_bboxes, alb_class_labels

In [13]:
transform = A.Compose([
    A.Blur(blur_limit=3, p=0.5),
    A.CLAHE(clip_limit=2, p=0.5),
    A.GaussianBlur(blur_limit=(3, 7), p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
    A.MultiplicativeNoise(multiplier=(0.9, 1.1), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Rotate(limit=10, p=0.5),
    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, p=0.5)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))


#transform = A.Compose([
#   A.Blur(blur_limit=7, p=0.5),
#    A.RandomBrightnessContrast(p=0.5),
#    A.RandomResizedCrop(size=(dataset_height, dataset_width), scale=(0.2, 1), p=1),
#    A.Rotate(limit=(-180,180), p=1),
#    A.Sharpen(p=0.5),
#    A.VerticalFlip(p=0.5)
#], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

In [14]:
source_folder = "filtered-data"
augmentation_folder = "augmented-data-2"
cont = 0

for image_path in train_images_paths:
  cont += 1
  print(cont, image_path)
  image = cv2.imread(image_path)
  labels_path = image_path.replace("images", "labels").replace(".jpg", ".txt")
  bboxes = load_bboxes(labels_path)
  
  alb_bboxes, alb_class_labels = bboxes_to_alb_bboxes(bboxes)
  alb_bboxes = fix_albumentation_range_errors(alb_bboxes)
  transformed = transform(image=image, bboxes=alb_bboxes, class_labels=alb_class_labels)
  bboxes = alb_bboxes_to_bboxes(transformed['bboxes'], transformed['class_labels'])
  
  #plot_image(transformed['image'], bboxes) #!COMMENT WHEN GENERATING DATASET
  
  copied_image_path = image_path.replace(source_folder, augmentation_folder)
  copied_labels_path = labels_path.replace(source_folder, augmentation_folder)
  augmented_image_path = image_path.replace(source_folder, augmentation_folder).replace(".jpg", "_augmented.jpg")
  augmented_labels_path = labels_path.replace(source_folder, augmentation_folder).replace(".txt", "_augmented.txt")

  shutil.copyfile(image_path, copied_image_path)
  cv2.imwrite(augmented_image_path, transformed['image'])

  shutil.copyfile(labels_path, copied_labels_path)
  
  new_lines = []
  for bbox in bboxes:
    id_class, x, y, w, h = bbox
    new_line = f"{id_class} {x} {y} {w} {h}\n"
    new_lines.append(new_line)
    
  with open(augmented_labels_path, 'w') as file:
        file.writelines(new_lines)

1 filtered-data/images/train/225uNbdMAuu4zCHfh.jpg
2 filtered-data/images/train/22HR9LhfZDGyprh9Y.jpg
3 filtered-data/images/train/22bTubsCWSDKawqDo.jpg
4 filtered-data/images/train/238zDcqk6aYAQcuqK.jpg
5 filtered-data/images/train/23amEQAHeMJ6Ljccr.jpg
6 filtered-data/images/train/23b5vFoRAApJQPH95.jpg
7 filtered-data/images/train/23vw5sCA9NhnHXC95.jpg
8 filtered-data/images/train/248hEHsd4wjLjC7Mr.jpg
9 filtered-data/images/train/24em6bwDPLDvmBRfr.jpg
10 filtered-data/images/train/24thDFiFrP3iFuP3a.jpg
11 filtered-data/images/train/25JQswyzHShXP5boy.jpg
12 filtered-data/images/train/25SqqbnNHGNQriDnp.jpg
13 filtered-data/images/train/268GjMZQFK43paqCn.jpg
14 filtered-data/images/train/26GLHtX6dHFJJKoM3.jpg
15 filtered-data/images/train/26Px32JEbwoXykfQY.jpg
16 filtered-data/images/train/27RZjNZGo8jvJE6a2.jpg
17 filtered-data/images/train/27djaaHZDhcSqESpB.jpg
18 filtered-data/images/train/27xjTJRXNXDeyDgQZ.jpg
19 filtered-data/images/train/2835M7kinHYtcToqX.jpg
20 filtered-data/imag